In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [136]:
data=pd.read_csv('Modified_SMSSpamCollection.csv')
data

,Label,Message
0,"""ham","Go until jurong point, crazy.. Available only ..."
1,"""ham","Ok lar... Joking wif u oni..."""
2,"""spam",Free entry in 2 a wkly comp to win FA Cup fina...
3,"""ham",U dun say so early hor... U c already then say...
4,"""ham","Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,"""spam",This is the 2nd time we have tried 2 contact u...
5570,"""ham","Will ü b going to esplanade fr home?"""
5571,"""ham","Pity, * was in mood for that. So...any other s..."
5572,"""ham",The guy did some bitching but I acted like i'd...


In [137]:
print(data.columns)

Index(['Label', 'Message'], dtype='object')


In [138]:
data['Label']=data['Label'].str[1:]
data['Message']=data['Message'].str[0:-1]
data

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will ü b going to esplanade fr home?
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...


In [139]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
Lemmatizer=WordNetLemmatizer()

In [140]:
data['Label']=data['Label'].map({'spam':0,'ham':1})
data

,Label,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,0,This is the 2nd time we have tried 2 contact u...
5570,1,Will ü b going to esplanade fr home?
5571,1,"Pity, * was in mood for that. So...any other s..."
5572,1,The guy did some bitching but I acted like i'd...


In [141]:
for i in range(data.shape[0]):
    document=data.iloc[i,1]
    words=document.split()
    words=[Lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    document=" ".join(words)
    data.iloc[i,1]=document
data    

,Label,Message
0,1,"Go jurong point, crazy.. Available bugis n gre..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry 2 wkly comp win FA Cup final tkts 2...
3,1,U dun say early hor... U c already say...
4,1,"Nah I think go usf, life around though"
...,...,...
5569,0,This 2nd time tried 2 contact u. U £750 Pound ...
5570,1,Will ü b going esplanade fr home?
5571,1,"Pity, * mood that. So...any suggestions?"
5572,1,The guy bitching I acted like i'd interested b...


In [142]:
X=data[['Message']]
y=data[['Label']]

In [143]:
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [144]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train

,Message
1350,FREE2DAY sexy St George's Day pic Jordan!Txt P...
5544,Armand say get as epsilon
1168,Lol I'm hot air balloon!
5551,"You know, wot people wear. T shirts, jumpers, ..."
5320,"Good morning, Love ... I go sleep wish great d..."
...,...
3772,"Hi, wlcome back, wonder got eaten lion somethi..."
5191,ree entry 2 weekly comp chance win ipod. Txt P...
5226,"""""OH FUCK. JUSWOKE UP IN A BED ON A BOATIN THE..."
5390,NOT MUCH NO FIGHTS. IT WAS A GOOD NITE!!


In [145]:
print(X_train.columns)

Index(['Message'], dtype='object')


In [146]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [147]:
cv=CountVectorizer(max_features=2500,ngram_range=(1,2))
tfidf=TfidfVectorizer(max_features=2500,ngram_range=(1,2))

In [148]:
X_train=cv.fit_transform(X_train['Message']).toarray()
X_test=cv.transform(X_test['Message']).toarray()

In [149]:
model=MultinomialNB()
model.fit(X_train,y_train)
model

C:\Users\mahap\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [150]:
y_pred=model.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [151]:
print(f"The accuracy score for spam ham classification by BOW is ",accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

The accuracy score for spam ham classification by BOW is  0.9847533632286996
[[150  11]
 [  6 948]]
              precision    recall  f1-score   support

           0       0.96      0.93      0.95       161
           1       0.99      0.99      0.99       954

    accuracy                           0.98      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.98      0.98      0.98      1115



In [152]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [153]:
X_train=tfidf.fit_transform(X_train['Message']).toarray()
X_test=tfidf.transform(X_test['Message']).toarray()
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [154]:
print(X_train.shape,X_test.shape)

(4459, 2500) (1115, 2500)


In [155]:
model.fit(X_train,y_train)
model

C:\Users\mahap\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [156]:
y_pred=model.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [157]:
print(f"The accuracy score for spam ham classification by TFIDF is ",accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

The accuracy score for spam ham classification by TFIDF is  0.9820627802690582
[[142  19]
 [  1 953]]
              precision    recall  f1-score   support

           0       0.99      0.88      0.93       161
           1       0.98      1.00      0.99       954

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [158]:
import gensim

In [159]:
from gensim.models import Word2Vec, KeyedVectors

In [160]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [161]:
X=data[['Message']]
Y=data[['Label']]

In [162]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train

,Message
1350,FREE2DAY sexy St George's Day pic Jordan!Txt P...
5544,Armand say get as epsilon
1168,Lol I'm hot air balloon!
5551,"You know, wot people wear. T shirts, jumpers, ..."
5320,"Good morning, Love ... I go sleep wish great d..."
...,...
3772,"Hi, wlcome back, wonder got eaten lion somethi..."
5191,ree entry 2 weekly comp chance win ipod. Txt P...
5226,"""""OH FUCK. JUSWOKE UP IN A BED ON A BOATIN THE..."
5390,NOT MUCH NO FIGHTS. IT WAS A GOOD NITE!!


In [163]:
#to train our word2Vec on our training dataset only we need to convert each senetence in X_train as list of words but dont chane X_train now
X_train_tokenized=[]
for doc in X_train['Message']:
    words=doc.split()
    X_train_tokenized.append(words)
len(X_train_tokenized)

4459

In [164]:
# Train Word2Vec on training data
model = Word2Vec(sentences=X_train_tokenized, vector_size=100, window=5, min_count=1, workers=4)

In [165]:

def sentence_to_vector(sentence, model):
    """Convert a sentence into a vector by averaging word vectors."""
    words = sentence.split()  # Tokenize the sentence
    words = [word for word in words if word in model.wv]  # Filter out-of-vocabulary words
    if not words:  # If no words are in the vocabulary, return a zero vector
        return np.zeros(model.vector_size)
    vectors = [model.wv[word] for word in words]  # Get word vectors
    return np.mean(vectors, axis=0)  # Average the word vectors


In [166]:
X_train = np.array([sentence_to_vector(sentence, model) for sentence in X_train])
X_test=np.array([sentence_to_vector(sentence,model) for sentence in X_test])

In [167]:
X_train

array([[ 0.00873349,  0.0021037 , -0.00331165,  0.00627201,  0.00922816,
         0.00184338, -0.00201888,  0.00240448, -0.00599408,  0.00655733,
         0.00077386, -0.00185367,  0.00395785,  0.00872701, -0.0023352 ,
        -0.00294908, -0.00918785, -0.00528908,  0.00440481, -0.00123058,
         0.00392132, -0.0024325 , -0.00171971,  0.00721555, -0.00920213,
         0.00306994, -0.00999507, -0.00875981,  0.00795581,  0.00226019,
         0.00866787, -0.00254184,  0.00505406,  0.00578227, -0.0046043 ,
        -0.00551439, -0.00742941, -0.00232834, -0.00011544, -0.00450566,
         0.00702299,  0.00600711,  0.00821294,  0.00992325, -0.00569876,
        -0.00122108,  0.00794991, -0.00873828,  0.001188  ,  0.00336989,
        -0.00113834,  0.00022087,  0.00015784,  0.00427007,  0.00516022,
         0.00194497, -0.00049082,  0.00887937, -0.00280139, -0.00611226,
         0.00429162,  0.00727875, -0.00422788,  0.00053459, -0.00080368,
         0.00403932, -0.00750546,  0.00229463,  0.0

In [168]:
X_test

array([[ 0.00873349,  0.0021037 , -0.00331165,  0.00627201,  0.00922816,
         0.00184338, -0.00201888,  0.00240448, -0.00599408,  0.00655733,
         0.00077386, -0.00185367,  0.00395785,  0.00872701, -0.0023352 ,
        -0.00294908, -0.00918785, -0.00528908,  0.00440481, -0.00123058,
         0.00392132, -0.0024325 , -0.00171971,  0.00721555, -0.00920213,
         0.00306994, -0.00999507, -0.00875981,  0.00795581,  0.00226019,
         0.00866787, -0.00254184,  0.00505406,  0.00578227, -0.0046043 ,
        -0.00551439, -0.00742941, -0.00232834, -0.00011544, -0.00450566,
         0.00702299,  0.00600711,  0.00821294,  0.00992325, -0.00569876,
        -0.00122108,  0.00794991, -0.00873828,  0.001188  ,  0.00336989,
        -0.00113834,  0.00022087,  0.00015784,  0.00427007,  0.00516022,
         0.00194497, -0.00049082,  0.00887937, -0.00280139, -0.00611226,
         0.00429162,  0.00727875, -0.00422788,  0.00053459, -0.00080368,
         0.00403932, -0.00750546,  0.00229463,  0.0

In [169]:
print(X_train.shape,X_test.shape)

(1, 100) (1, 100)
